In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Paths to your dataset directories
train_dir = 'data/colored_images'

# Image Data Generator for data augmentation and rescaling
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.3,  # 30% split for validation and testing combined
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Creating training, validation, and testing generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Set as training data (70% of the original data)
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.5  # 50% of the remaining 30% for validation and testing
)

val_generator = validation_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',  # Set as validation data (15% of the original data)
    shuffle=False
)

test_generator = validation_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',  # Set as testing data (15% of the original data)
    shuffle=False
)

# Define the CNN model
def build_model(activation='relu'):
    model = Sequential([
        Conv2D(32, (3, 3), activation=activation, input_shape=(224, 224, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation=activation),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation=activation),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(256, activation=activation),
        Dropout(0.5),
        Dense(5, activation='softmax')  # 5 classes
    ])
    
    return model

# Build the model
model = build_model(activation='relu')

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stopping, model_checkpoint]
)

# Evaluate the model on the validation data
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation accuracy: {val_acc:.4f}")
print(f"Validation loss: {val_loss:.4f}")

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc:.4f}")
print(f"Test loss: {test_loss:.4f}")


Found 2566 images belonging to 5 classes.
Found 1829 images belonging to 5 classes.
Found 1833 images belonging to 5 classes.


/Users/mithindev/Desktop/COLLEGE/SEM-5/DL/CODE/venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=best_model.h5